# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [18]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fuzha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fuzha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\fuzha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [38]:
# import libraries
import re
from sqlalchemy import create_engine
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import pickle


# i.e. creating a cutomize transformer
# import the libraries
from sklearn.base import BaseEstimator, TransformerMixin
# sentence tozenize
from nltk import sent_tokenize
# tokenizing into word and tag of a speech
from nltk import pos_tag

In [20]:
# load data from database
engine = create_engine('sqlite:///disaster_messages.db')
df = pd.read_sql_table('disaster_messages', engine)
df.head(n=5)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
X = df.message
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [22]:
def tokenize_text(text):
    #print(text)
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # find all the urls
    find_url = re.findall(url_regex, text)
    # replace the urls with 'urlplaceholder'
    for url in find_url:
        text = text.replace(url, 'urlplaceholder')
    #print(text)
    # tokenize the text
    words = word_tokenize(text)
    # lemmatize the words
    # make all the characters lower and remove the punctuations and whitespaces
    clean_text = [WordNetLemmatizer().lemmatize(word).lower().strip() for word in words]

    return clean_text


In [27]:
# for improving the model a custom transformation is defined below.
class StartingVerbExtrator(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        #tokernize the sentence
        tokenize_sentence = sent_tokenize(text)

        # tokenize each sentence into words and tag
        for sentence in tokenize_sentence:
            pos_tags = pos_tag(tokenize_text(sentence))
            # get the first word and part of speech tag by indexing the pos_tag
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply the custom starting_verb function to all the X values
        X_tagged = pd.Series(X).apply(self.starting_verb)
        # make a dataframe of the result
        return pd.DataFrame(X_tagged)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [28]:
ml_pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize_text)),
            ('tfidf', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtrator())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [29]:
# split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# train the pipeline
ml_pipeline.fit(X_train, Y_train)



Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize_text at 0x0000014FB2F3AB80>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtrator())])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [30]:
y_pred = ml_pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [31]:
len(Y.columns)

36

In [32]:
for i in range(0, len(Y.columns)):
    print(Y.columns[i])
    print(classification_report(Y_test.iloc[:, i].values, y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.76      0.27      0.40      1501
           1       0.81      0.97      0.88      5000
           2       0.57      0.08      0.13        53

    accuracy                           0.81      6554
   macro avg       0.71      0.44      0.47      6554
weighted avg       0.80      0.81      0.77      6554

request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5422
           1       0.90      0.44      0.59      1132

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6514
           1       0.00      0.00      0.00        40

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6228
           1       0.74      0.07      0.13       326

    accuracy                           0.95      6554
   macro avg       0.85      0.53      0.55      6554
weighted avg       0.94      0.95      0.93      6554

buildings
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6215
           1       0.79      0.04      0.08       339

    accuracy                           0.95      6554
   macro avg       0.87      0.52      0.53      6554
weighted avg       0.94      0.95      0.93      6554

electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6409
           1       1.00      0.03      0.05       145

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.52      6554
weighted avg       0.98      0.98      0.97      655

c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metr

### 6. Improve your model
Use grid search to find better parameters. 

In [33]:
# find the parameters available for grid search
available_parameters = ml_pipeline.get_params()
print(available_parameters)

{'memory': None, 'steps': [('features', FeatureUnion(transformer_list=[('text_pipeline',
                                Pipeline(steps=[('vect',
                                                 CountVectorizer(tokenizer=<function tokenize_text at 0x0000014FB2F3AB80>)),
                                                ('tfidf',
                                                 TfidfTransformer())])),
                               ('starting_verb', StartingVerbExtrator())])), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))], 'verbose': False, 'features': FeatureUnion(transformer_list=[('text_pipeline',
                                Pipeline(steps=[('vect',
                                                 CountVectorizer(tokenizer=<function tokenize_text at 0x0000014FB2F3AB80>)),
                                                ('tfidf',
                                                 TfidfTransformer())])),
                               ('starting_verb', StartingVer

In [34]:

parameters = {
    #'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__n_estimators': [50, 100]
}

cv = GridSearchCV(ml_pipeline, param_grid=parameters, n_jobs=3) # verbose=2,

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [35]:
cv.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize_text at 0x0000014FB2F3AB80>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtrator())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=3, param_grid={'clf__estimat

In [36]:
# find the y_pred for the best model
y_pred_optimized =cv.best_estimator_ .predict(X_test)

# get the classification report for the optimized model
for i in range(0, len(Y.columns)):
    print(Y.columns[i])
    print(classification_report(Y_test.iloc[:, i].values, y_pred_optimized[:, i]))

related
              precision    recall  f1-score   support

           0       0.76      0.28      0.41      1501
           1       0.81      0.97      0.89      5000
           2       0.57      0.15      0.24        53

    accuracy                           0.81      6554
   macro avg       0.72      0.47      0.51      6554
weighted avg       0.80      0.81      0.77      6554

request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5422
           1       0.89      0.44      0.59      1132

    accuracy                           0.89      6554
   macro avg       0.89      0.72      0.77      6554
weighted avg       0.89      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6514
           1       0.00      0.00      0.00        40

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5715
           1       0.69      0.02      0.05       839

    accuracy                           0.87      6554
   macro avg       0.78      0.51      0.49      6554
weighted avg       0.85      0.87      0.82      6554

infrastructure_related
              precision    recall  f1-score   support

           0       0.93      1.00      0.97      6117
           1       0.33      0.00      0.00       437

    accuracy                           0.93      6554
   macro avg       0.63      0.50      0.48      6554
weighted avg       0.89      0.93      0.90      6554

transport
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6228
           1       0.74      0.09      0.16       326

    accuracy                           0.95      6554
   macro avg       0.85      0.54      0.57      6554
weighted avg       0.94      0.95      0.

c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fuzha\anaconda3\lib\site-packages\sklearn\metr

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# above

### 9. Export your model as a pickle file

In [40]:
with open('model.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.